In [1]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

170508288/170498071 [==============================] - 4s 0us/step


In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 10),
 (10000, 32, 32, 3),
 (10000, 10),
 (10000, 32, 32, 3),
 (10000, 10))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def alexnet(activation):
    AlexNet = Sequential()

    AlexNet.add(Conv2D(filters=96, input_shape=(32, 32, 3), kernel_size=(11, 11), strides=(4, 4), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Flatten())
    AlexNet.add(Dense(4096, input_shape=(32, 32, 3)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(10))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))

    return AlexNet

In [7]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = alexnet('relu')

batch_size = 128
epochs = 50

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 72s 80ms/step - loss: 2.1473 - accuracy: 0.2353 - val_loss: 1.8497 - val_accuracy: 0.3462
Epoch 2/50
313/313 [==============================] - 22s 71ms/step - loss: 1.6361 - accuracy: 0.4194 - val_loss: 1.5370 - val_accuracy: 0.4640
Epoch 3/50
313/313 [==============================] - 24s 76ms/step - loss: 1.4846 - accuracy: 0.4855 - val_loss: 1.3768 - val_accuracy: 0.5276
Epoch 4/50
313/313 [==============================] - 23s 74ms/step - loss: 1.3740 - accuracy: 0.5290 - val_loss: 1.5469 - val_accuracy: 0.4505
Epoch 5/50
313/313 [==============================] - 22s 70ms/step - loss: 1.2930 - accuracy: 0.5609 - val_loss: 1.4445 - val_accuracy: 0.4909
Epoch 6/50
313/313 [==============================] - 23s 75ms/step - loss: 1.2188 - accuracy: 0.5825 - val_loss: 1.4059 - val_accuracy: 0.5095
Epoch 7/50
313/313 [==============================] - 22s 72ms/step - loss: 1.1698 - accuracy: 0.6012 - val_loss: 1.8661 - val_accuracy:

In [8]:
print(history.history)
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test,axis=1)

print(y_pred.shape)
print(y_true.shape)

print(np.sum(y_pred == y_true) / y_pred.shape[0])

{'loss': [1.9315803050994873, 1.5992205142974854, 1.4562040567398071, 1.3618193864822388, 1.2806966304779053, 1.2123916149139404, 1.1589703559875488, 1.1078267097473145, 1.0610688924789429, 1.0178948640823364, 0.9778406023979187, 0.9409791231155396, 0.9056694507598877, 0.8719632029533386, 0.8419684767723083, 0.8109716176986694, 0.7838069200515747, 0.7466331720352173, 0.7198708057403564, 0.6957352161407471, 0.6707767844200134, 0.6373634338378906, 0.6145175099372864, 0.5908297896385193, 0.5643681883811951, 0.5395238995552063, 0.5174742937088013, 0.49186405539512634, 0.47206082940101624, 0.45157426595687866, 0.4297679662704468, 0.4099060595035553, 0.39276614785194397, 0.378933310508728, 0.35171109437942505, 0.33860132098197937, 0.31415855884552, 0.30760657787323, 0.28890904784202576, 0.2784215211868286, 0.2650837004184723, 0.2464938908815384, 0.23266291618347168, 0.21967224776744843, 0.21205168962478638, 0.19953615963459015, 0.19468005001544952, 0.18626831471920013, 0.16691702604293823, 0